In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x107ea5590>)

In [ ]:
#Split words, convert them into ids when encoding
#While decoding, convert them back to words with vocabulary
import re
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        tokens = " ".join([self.int_to_str[index] for index in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', tokens)
        return text

In [ ]:
# Add <|unknown|> tag to handle unknown word
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int else '<|unknown|>' for item in preprocessed]
        ids = [self.str_to_int[x] for x in preprocessed]
        return ids
    
    def decode(self, ids):
        tokens = " ".join([self.int_to_str[index] for index in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', tokens)
        return text

In [5]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
text = "Hello, nice to meet you!<|end|> How is it going?"
tokens = tokenizer.encode(text=text, allowed_special={'<|end|>'})
print(tokens)

string = tokenizer.decode(tokens)
print(string)

[15496, 11, 3621, 284, 1826, 345, 0, 27, 91, 437, 91, 29, 1374, 318, 340, 1016, 30]
Hello, nice to meet you!<|end|> How is it going?


In [6]:
unknown_word = 'Akwirw ler'
unknown_word_tokens = tokenizer.encode(unknown_word)
print(unknown_word_tokens)
unknown_word_decoded = tokenizer.decode(unknown_word_tokens)
print(unknown_word_decoded)


[33901, 86, 343, 86, 300, 263]
Akwirw ler


In [7]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [8]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input = []
        self.target = []
        
        token_ids = tokenizer.encode(text)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input.append(torch.tensor(input_chunk))
            self.target.append(torch.tensor(target_chunk))
            
            
    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, index):
        return self.input[index], self.target[index]

In [9]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128,
                         shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset=dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader



In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, 
                                  max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
input, target = next(data_iter)
print(input)
print(input.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])
